Given input , return the balanced equation.
Eample input string: ” K Mn O 4 + H 2 O 2 = Mn O 2 + H 2 O + O 2 + K O H ”. Assume input
has spaces separating every element name and numbers as shown, also for ‘+’ and ‘=’.
Example output: ” 2 K Mn O 4 + 3 H 2 O 2 = 2 Mn O 2 + 2 H 2 O + 3 O 2 + 2 K O H ” and ” [
-2,- 3, 2, 3, 3, 2 ]”, i.e. output string for balanced chemical equation and the vector of stochiometric
coefficients with negative values for reactants.
HINT: (a) For the above reaction, atomList is ’K’, ’Mn’, ’O’, ’H’
(b) chemVectors for ‘ K Mn O 4’ is [1,1,4,0] and ’ K O H ’ is [1,0,1,1]
(c) construct matrix equation for chemical equation.
1

In [1]:
import numpy as np

In [14]:
equation = "K Mn O 4 + H 2 O 2 = Mn O 2 + O 2 + H 2 O + K O H"

In [16]:
import re
from sympy import Matrix, lcm

elementList = []
elementMatrix = []

if not equation:
    equation = input("Enter the equation: ")
    
equation = equation.replace(" ", "")
reactants, products = equation.split("=")

def parseFormula(formula):
    elementList = re.findall(r"")

<>:43: SyntaxWarning: invalid escape sequence '\('
<>:46: SyntaxWarning: invalid escape sequence '\)'
<>:51: SyntaxWarning: invalid escape sequence '\['
<>:43: SyntaxWarning: invalid escape sequence '\('
<>:46: SyntaxWarning: invalid escape sequence '\)'
<>:51: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_186044/2833438180.py:43: SyntaxWarning: invalid escape sequence '\('
  segments = re.split('(\([A-Za-z0-9]*\)[0-9]*)', compound)
/tmp/ipykernel_186044/2833438180.py:46: SyntaxWarning: invalid escape sequence '\)'
  segment = re.split('\)([0-9]*)', segment)
/tmp/ipykernel_186044/2833438180.py:51: SyntaxWarning: invalid escape sequence '\['
  findElements(re.sub('(\[|\])', '', segment), index, multiplier, side)


Please input your reactants. This is case sensitive.
Your input should look like: H2 + O2


Please input your products. This is case sensitive.
Your input should look like: H2O
2KMO4 + 1H2O2 -> 2MO2 + 2KOH + 2O2 + 0H2O
